Тема - Предсказание спортивных результатов по физиологическим параметрам

In [1]:
# data_loader.py

from sklearn.datasets import load_linnerud
from sklearn.utils import Bunch
import pandas as pd

class DataLoader:
    def load(self):
        """Загрузка данных из Linnerud датасет.
        
        :return: Физиологические признаки и спортивные результаты.
        """
        data: Bunch = load_linnerud() # Загружает датасет Linnerud как Bunch-объект
        x = pd.DataFrame(data.target, columns=data.target_names) # Физиологические признаки (X)
        y = pd.DataFrame(data.data, columns=data.feature_names) # Спортивные результаты (Y)
        return x, y

In [2]:
# data_preprocessor.py

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import RegressorMixin

class DataPreprocessor:
    def __init__(self):
        self.imputer = SimpleImputer(strategy='mean') # Заполнение пропусков средним значением
        self.scaler = StandardScaler() # Масштабирование: среднее = 0, стандартное отклонение = 1

    def preprocess(self, df: pd.DataFrame) -> pd.DataFrame:
        df_imputed = self.imputer.fit_transform(df) # Заполняет пропуски
        df_scaled = self.scaler.fit_transform(df_imputed) # Масштабирует признаки
        return pd.DataFrame(df_scaled, columns=df.columns)

In [3]:
# trainer.py

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
import pandas as pd

class Trainer:
    def __init__(self):
        self.model = LinearRegression() # Инициализация модели линейной регрессии

    def split(self, X, y, test_size=0.3, random_state=42):
        return train_test_split(X, y, test_size=test_size, random_state=random_state) # Делит данные на обучающую и тестовую выборки (test_size=0.3 → 30% данных идут в тест, random_state=42 → фиксирует случайность для воспроизводимости)

    def fit(self, x, y):
        self.model.fit(x, y) # Обучение модели на признаках x и целевой переменной y

    def predict(self, x):
        return self.model.predict(x) # Предсказание на новых данных x

    def evaluate(self, y_true, y_pred):
        return pd.DataFrame({
            'R2': [r2_score(y_true, y_pred)], # Коэффициент детерминации: насколько хорошо модель объясняет данные
            'MSE': [mean_squared_error(y_true, y_pred)] # Среднеквадратичная ошибка: насколько сильно модель ошибается
        })

    def get_model(self):
        return self.model

In [4]:
# data_visualizer.py

import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

class DataVisualizer:
    def plot_predictions(self, y_true: pd.Series, y_pred: pd.Series, title: str = "Сравнение фактических и предсказанных значений подтягиваний"):
        df = pd.DataFrame({'Реальные': y_true, 'Предсказанные': y_pred})
        fig = px.scatter(df, x='Реальные', y='Предсказанные', title=title, opacity=0.7)
        fig.add_shape(type='line',
                      x0=df['Реальные'].min(), y0=df['Реальные'].min(),
                      x1=df['Реальные'].max(), y1=df['Реальные'].max(),
                      line=dict(color='red', dash='dash'))
        return fig

    def plot_target_distribution(self, y: pd.Series, title: str = "Распределение подтягиваний"):
        """Гистограмма распределения целевой переменной (Chins).
        """
        fig = px.histogram(y, nbins=10, title=title,
                       labels={'value': 'Количество подтягиваний'},
                       color_discrete_sequence=['skyblue'])
        fig.update_layout(yaxis_title='Частота')
        return fig

    def plot_feature_vs_target(self, x: pd.DataFrame, y: pd.Series, feature: str):
        """Точечная диаграмма зависимости исходного признака от целевой переменной (Chins).
        """
        df = pd.DataFrame({f"Признак «{feature}»": x[feature], 'Количество подтягиваний': y})
        fig = px.scatter(df, x=f"Признак «{feature}»", y='Количество подтягиваний',
                        title=f"Зависимость признака «{feature}» от подтягиваний", opacity=0.7)
        return fig

    def plot_correlation_heatmap(self, x: pd.DataFrame, y: pd.Series, title: str = "Корреляции между физическими признаками и подтягиванием"):
        """Тепловая карта корреляций признаков и целевой переменной (Chins).
        """
        df = x.copy()
        df['Chins'] = y
        corr = df.corr().round(2)
        fig = ff.create_annotated_heatmap(z=corr.values,
                                        x=corr.columns.tolist(),
                                        y=corr.index.tolist(),
                                        colorscale='RdBu',
                                        showscale=True,
                                        reversescale=True,
                                        annotation_text=corr.values.astype(str))
        fig.update_layout(title_text=title)
        return fig
    
    def plot_metrics_table(self, results: pd.DataFrame, title: str = "Оценка модели"):
        fig = go.Figure(data=[go.Table(
        header=dict(values=list(results.columns),
                    fill_color='lightgrey',
                    align='left'),
        cells=dict(values=[results[col].tolist() for col in results.columns],
                   fill_color='white',
                   align='left'))
        ])
        fig.update_layout(title=title)
        return fig

In [5]:
# training.py

from data_loader import DataLoader
from data_preprocessor import DataPreprocessor
from trainer import Trainer
from data_visualizer import DataVisualizer
import pandas as pd
import panel as pn
pn.extension('plotly')

# 1. Загрузка
x_raw, y_raw = DataLoader().load()
target_column = 'Chins' # Chins - Количество подтягиваний
y = y_raw[target_column]

# 2. Предобработка
x = DataPreprocessor().preprocess(x_raw)

# 3. Разделение, Обучение, Предсказание и Оценка учителем
trainer = Trainer()
x_train, x_test, y_train, y_test = trainer.split(x, y)
trainer.fit(x_train, y_train)
y_pred = trainer.predict(x_test)
metrics = trainer.evaluate(y_test, y_pred)

# 4. Визуализация
visualizer = DataVisualizer()
metrics = metrics.T.reset_index()
metrics.columns = ['Метрика', 'Значение']
tabs = pn.Tabs(
    ("Предсказания", pn.pane.Plotly(visualizer.plot_predictions(pd.Series(y_test, name="Реальные"), pd.Series(y_pred, name='Предсказанные')))),
    ("Распределение", pn.pane.Plotly(visualizer.plot_target_distribution(y))),
    ("Зависимость признака", pn.pane.Plotly(visualizer.plot_feature_vs_target(x, y, feature='Weight'))),
    ("Корреляции", pn.pane.Plotly(visualizer.plot_correlation_heatmap(x, y))),
    ("Оценка модели", visualizer.plot_metrics_table(metrics))
)

# pn.serve(tabs, show=True, title="Анализ подтягиваний", port=5006)

In [6]:
tabs

Tabs
    [0] Plotly(Figure)
    [1] Plotly(Figure)
    [2] Plotly(Figure)
    [3] Plotly(Figure)
    [4] Plotly(Figure, name='Оценка модели')

Таким образом, я построил систему, которая:
- Загружает и обрабатывает физиологические данные (Weight, Waist, Pulse).
- Предсказывает спортивный результат — количество подтягиваний (Chins).
- Оценивает качество модели.
- Визуализирует связи между признаками и целевой переменной.

Что дали метрики:
- R² (коэффициент детерминации) — показывает, насколько хорошо модель объясняет вариацию в подтягиваниях → Чем ближе к 1, тем лучше
- MSE (среднеквадратичная ошибка) — показывает, насколько сильно модель ошибается → Чем меньше, тем точнее

Эти метрики дали численную оценку качества модели и позволили сравнивать её с альтернативами.

Что делают графики:
- График plot_predictions показывает, насколько предсказанные значения подтягиваний совпадают с реальными. Он позволяет оценить точность модели, выявить наличие смещения и понять, насколько сильно разбросаны ошибки. Модель в целом хорошо предсказывает количество подтягиваний, но есть отклонения от идеальной диагонали. Это говорит о наличии ошибок, особенно при крайних значениях, и возможном смещении.
- График plot_target_distribution отражает распределение количества подтягиваний в выборке. С его помощью можно определить, сбалансированы ли данные, есть ли выбросы или перекос в сторону определённых значений. Распределение подтягиваний неравномерное: большинство наблюдений сосредоточено в определённом диапазоне. Это может влиять на обучение модели и требует внимания к балансировке. 
- График plot_feature_vs_target иллюстрирует зависимость одного признака (например, веса или пульса) от целевой переменной. Он помогает понять, насколько признак полезен для предсказания и какова форма этой зависимости — линейная, нелинейная или отсутствующая. Некоторые признаки (например, вес) демонстрируют заметную зависимость от количества подтягиваний. Это подтверждает, что они информативны и полезны для предсказания.
- График plot_correlation_heatmap визуализирует корреляции между всеми признаками и количеством подтягиваний. Он позволяет выявить наиболее влиятельные переменные, а также определить наличие мультиколлинеарности — признаков, сильно коррелирующих друг с другом. Видны сильные и слабые корреляции между признаками и Chins. Например, если Waist или Pulse слабо связаны с подтягиваниями, их влияние на модель может быть ограничено. Также можно заметить признаки, которые коррелируют друг с другом — это сигнал к проверке на мультиколлинеарность.